In [1]:
import pandas as pd
import math
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
import numpy as np

In [2]:
df = pd.read_csv('terrenos.csv',header=0,encoding = 'unicode_escape',error_bad_lines=False)
crs = {'init' : 'epsg:4326'}
df.head()

b'Skipping line 2618: expected 7 fields, saw 11\nSkipping line 15259: expected 7 fields, saw 9\nSkipping line 23843: expected 7 fields, saw 9\n'


,Title,Address,Descripcion,Precio,TamaÃ±o,Latitud,Longitud
0,LagunasInterior . Punta TiburÃ³n Residencial; ...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,$ 769;500,180 mÂ²,19.067293,-96.080747
1,La Escondida Residencial â Terrenos,Octava Avenida y Veracruz||RincÃ³n de La Sierr...,La Escondida ResidencialResidencias y Lotes de...,$ 711;000,140 mÂ²,25.641346,-100.198342
2,El Molino Residencial & Golf,Paseos El Molino # 302; LeÃ³n; Gto||Balcones d...,El Molino; Residencial & GolfEs un impresionan...,$ 845;719,250 mÂ²,21.185639,-101.699237
3,Club Cenote Residencial,Ruta de los Cenotes Km 19 DesviaciÃ³n Delirios...,Club Cenote ResidencialAmplios Terrenos en el ...,$ 1;039;000,800 mÂ²,20.847873,-87.220608
4,Punta TiburÃ³n Residencial; Marina & Golf - Bo...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,$ 723;672,180 mÂ²,19.067293,-96.080747


In [3]:
geometry = [Point(xy) for xy in zip(df['Longitud'], df['Latitud'])]

In [4]:
geo_df = gpd.GeoDataFrame(df,
                         crs = crs,
                         geometry = geometry)
geo_df.head()

,Title,Address,Descripcion,Precio,TamaÃ±o,Latitud,Longitud,geometry
0,LagunasInterior . Punta TiburÃ³n Residencial; ...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,$ 769;500,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729)
1,La Escondida Residencial â Terrenos,Octava Avenida y Veracruz||RincÃ³n de La Sierr...,La Escondida ResidencialResidencias y Lotes de...,$ 711;000,140 mÂ²,25.641346,-100.198342,POINT (-100.19834 25.64135)
2,El Molino Residencial & Golf,Paseos El Molino # 302; LeÃ³n; Gto||Balcones d...,El Molino; Residencial & GolfEs un impresionan...,$ 845;719,250 mÂ²,21.185639,-101.699237,POINT (-101.69924 21.18564)
3,Club Cenote Residencial,Ruta de los Cenotes Km 19 DesviaciÃ³n Delirios...,Club Cenote ResidencialAmplios Terrenos en el ...,$ 1;039;000,800 mÂ²,20.847873,-87.220608,POINT (-87.22061 20.84787)
4,Punta TiburÃ³n Residencial; Marina & Golf - Bo...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,$ 723;672,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729)


In [5]:
precio = [str(row).replace(";","").replace("$","").replace(' ','').replace('|','') for  row in geo_df['Precio']]
tipo_cambio=19.33
precio = [float(row.replace('US',''))*tipo_cambio if 'US' in row else float(row) for row in precio]
geo_df['Precio'] = precio
geo_df['Tamano'] = [float(str(row).replace(' mÂ²','')) for row in geo_df['TamaÃ±o']]
#precio.isna().sum()
geo_df['Precio/Tam']=[row['Precio']/row['Tamano'] for idx,row in geo_df.iterrows()]
geo_df.head()

,Title,Address,Descripcion,Precio,TamaÃ±o,Latitud,Longitud,geometry,Tamano,Precio/Tam
0,LagunasInterior . Punta TiburÃ³n Residencial; ...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,769500.0,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729),180.0,4275.000000
1,La Escondida Residencial â Terrenos,Octava Avenida y Veracruz||RincÃ³n de La Sierr...,La Escondida ResidencialResidencias y Lotes de...,711000.0,140 mÂ²,25.641346,-100.198342,POINT (-100.19834 25.64135),140.0,5078.571429
2,El Molino Residencial & Golf,Paseos El Molino # 302; LeÃ³n; Gto||Balcones d...,El Molino; Residencial & GolfEs un impresionan...,845719.0,250 mÂ²,21.185639,-101.699237,POINT (-101.69924 21.18564),250.0,3382.876000
3,Club Cenote Residencial,Ruta de los Cenotes Km 19 DesviaciÃ³n Delirios...,Club Cenote ResidencialAmplios Terrenos en el ...,1039000.0,800 mÂ²,20.847873,-87.220608,POINT (-87.22061 20.84787),800.0,1298.750000
4,Punta TiburÃ³n Residencial; Marina & Golf - Bo...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,723672.0,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729),180.0,4020.400000


In [6]:
geo_df[geo_df['Precio/Tam']==0]

,Title,Address,Descripcion,Precio,TamaÃ±o,Latitud,Longitud,geometry,Tamano,Precio/Tam
20198,LOTE EN VENTA FRENTE A LA MARINA EN PUERTO AVE...,Carretera Federal km 307 Puerto Juarez -Chetum...,MagnÃ­fico lote con frente de marina en uno de...,0.0,750 mÂ²,20.504,-87.226,POINT (-87.22600 20.50400),750.0,0.0


In [7]:
geo_df = geo_df.dropna()
geo_df = geo_df[geo_df['Precio/Tam']!=0]
np.log(geo_df['Precio/Tam']).hist()
geo_df.shape
[geo_df[geo_df['Precio/Tam']<150].shape, geo_df[(np.log(geo_df['Precio/Tam'])<5)].shape, 
 geo_df[geo_df['Precio/Tam']>20000].shape, geo_df[(np.log(geo_df['Precio/Tam'])>10)].shape]

[(4793, 10), (4784, 10), (1407, 10), (1283, 10)]

In [8]:
geo_df1 = geo_df[(np.log(geo_df['Precio/Tam'])>=5) & (np.log(geo_df['Precio/Tam'])<=10)]
max(geo_df1['Precio/Tam'])

22013.274336283186

In [9]:
geo_df=geo_df1
geo_df.shape

(23248, 10)

In [10]:
geo_df.head()

,Title,Address,Descripcion,Precio,TamaÃ±o,Latitud,Longitud,geometry,Tamano,Precio/Tam
0,LagunasInterior . Punta TiburÃ³n Residencial; ...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,769500.0,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729),180.0,4275.000000
1,La Escondida Residencial â Terrenos,Octava Avenida y Veracruz||RincÃ³n de La Sierr...,La Escondida ResidencialResidencias y Lotes de...,711000.0,140 mÂ²,25.641346,-100.198342,POINT (-100.19834 25.64135),140.0,5078.571429
2,El Molino Residencial & Golf,Paseos El Molino # 302; LeÃ³n; Gto||Balcones d...,El Molino; Residencial & GolfEs un impresionan...,845719.0,250 mÂ²,21.185639,-101.699237,POINT (-101.69924 21.18564),250.0,3382.876000
3,Club Cenote Residencial,Ruta de los Cenotes Km 19 DesviaciÃ³n Delirios...,Club Cenote ResidencialAmplios Terrenos en el ...,1039000.0,800 mÂ²,20.847873,-87.220608,POINT (-87.22061 20.84787),800.0,1298.750000
4,Punta TiburÃ³n Residencial; Marina & Golf - Bo...,Km. 4.5 Carretera Federal Boca del RÃ­o - AntÃ...,Punta tiburÃ³n residencial se encuentra en la ...,723672.0,180 mÂ²,19.067293,-96.080747,POINT (-96.08075 19.06729),180.0,4020.400000


In [11]:
geo_df.to_file(driver='ESRI Shapefile', filename='conjunto_de_datos/Terrenos/terrenos.shp')